# Exercise: Deforestation Detection with Foundation Models

**Notice: This Notebook is not yet complete. Currently only the Unsupervised Part of the Notebook can reliably be used!**

In this exercise, you'll apply foundation model embeddings to detect deforestation in the Amazon rainforest.

## Learning Objectives
- Load and visualize satellite imagery for a deforestation hotspot
- Extract and explore foundation model embeddings
- Apply unsupervised clustering to identify land cover patterns
- Train a supervised classifier to detect deforestation
- Evaluate model performance and interpret results

## Background: Why Deforestation Detection?

Deforestation in the Amazon has accelerated in recent years, with significant impacts on:
- **Climate Change**: The Amazon stores ~150-200 billion tons of carbon
- **Biodiversity**: Home to 10% of Earth's species
- **Indigenous Communities**: Millions depend on the forest
- **Regional Climate**: Affects rainfall patterns across South America

Traditional monitoring requires extensive field surveys. Foundation models enable:
- **Rapid Detection**: Identify changes quickly from satellite imagery
- **Scalable Monitoring**: Cover vast areas efficiently
- **Early Warning**: Detect clearing before it becomes widespread

## About the Study Area

We'll focus on a region at the border of **Pará and Mato Grosso** states in Brazil, which has experienced significant deforestation due to:
- Cattle ranching expansion
- Soy cultivation
- Logging operations
- Road construction opening new areas (particularly along BR-163 highway)

This area is part of the "arc of deforestation" - the agricultural frontier advancing into the Amazon. You'll see a mix of intact forest, recent clearings, and agricultural land.

In this notebook, when we talk about deforestation, we refer to any kind of observable canopy loss. Sometimes it might be unclear what exactly the reason for this was (e.g wildfire, human intervention, other...).

## Setup

First, install required packages and import helper functions.

In [ ]:
# Install dependencies
%pip install numpy pandas matplotlib rasterio seaborn xarray pyproj dask rioxarray pystac-client planetary-computer scikit-learn pyarrow tqdm scipy leafmap -q

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
import pandas as pd
import seaborn as sns

def in_colab():
    try:
        import google.colab
        return True
    except ImportError:
        return False

if in_colab():
    !git clone https://github.com/rohansaw/FoundationModels4EO.git
    %cd FoundationModels4EO
else:
    print("Running locally - skipping git clone")

# Import helper functions
from geo_helpers import (
    load_sentinel2_rgb_timeseries,
    load_foundation_model_embeddings,
    create_embedding_rgb,
    compute_clusters,
    predict_on_embeddings
)

from viz_helpers import (
    plot_embeddings_rgb,
    plot_clustering_results,
    plot_classification_results,
    show_study_area_map,
    plot_classification_vs_clustering,
    plot_rgb_comparison,
    plot_rgb_with_change_magnitude,
    plot_rgb_with_clusters
)

## Part 1: Define Study Area and Explore Imagery

**TODO #1**: Define the bounding box for your study area in Brazil.

We've selected a region at the border of Pará and Mato Grosso states, known for active deforestation. The coordinates are provided below.

**Your Task**: 
- Run the cell to visualize the study area on an interactive map
- Examine the region and note:
  - Where do you see intact forest?
  - Where do you see cleared areas?
  - Can you identify roads or agricultural fields?

In [ ]:
# Study area in Pará/Mato Grosso, Brazil - active deforestation zone
# Centered around lat: -7.03416, lon: -52.46191
LON_MIN, LON_MAX = -52.48, -52.36
LAT_MIN, LAT_MAX = -7.06, -6.93
# Specify two years between 2018 and 2025: baseline_year and target_year
baseline_year = 2019  # baseline (before) year - change as needed (2018-2025)
target_year = 2024    # target (after) year - change as needed (2018-2025)

# Validate allowed year range (2018-2025)
for y in (baseline_year, target_year):
    if y < 2018 or y > 2025:
        raise ValueError(f"Year {y} is outside the allowed range 2018-2025")

# Visualize the study area
print("Study Area: Pará/Mato Grosso, Brazil")
show_study_area_map(LON_MIN, LON_MAX, LAT_MIN, LAT_MAX)

### Load Satellite Imagery - Compare 2019 vs 2024

**TODO #2**: Load Sentinel-2 RGB imagery to detect deforestation over time.

We'll compare imagery from 2019 (before recent deforestation) and 2024 (current state) to visualize changes.

The dry season (May-September) in the Amazon is best for detecting deforestation because:
- Less cloud cover
- Cleared areas are more visible
- Burning activity peaks during this period

**Your Task**:
- Run the cell to load 2 images from 2019 and 2 from 2024
- Compare the before/after imagery
- Can you spot deforestation that occurred between 2019 and 2024?

In [ ]:
# Load Sentinel-2 RGB imagery - comparing baseline_year vs target_year
# We'll load 2 images from each year during the dry season (example: June and August)

print(f"Loading imagery from {baseline_year} and {target_year} for comparison...")

# Baseline imagery (2 images from dry season)
rgb_baseline = load_sentinel2_rgb_timeseries(
    LON_MIN, LON_MAX, LAT_MIN, LAT_MAX, baseline_year, [8]  # June and August
)

# Target imagery (2 images from dry season)
rgb_target = load_sentinel2_rgb_timeseries(
    LON_MIN, LON_MAX, LAT_MIN, LAT_MAX, target_year, [8]  # June and August
)

print(f"Baseline image shape: {rgb_baseline[list(rgb_baseline.keys())[0]].shape}")
print(f"Target image shape: {rgb_target[list(rgb_target.keys())[0]].shape}")


# Visualize side-by-side for easy comparison
plot_rgb_comparison(rgb_baseline, rgb_target, "Pará/Mato Grosso, Brazil", baseline_year, target_year)

## Part 2: Load and Visualize Foundation Model Embeddings

**TODO #3**: Load the Alpha Earth embeddings for your study area.

Instead of working directly with raw satellite imagery (which has many spectral bands and temporal dimensions), we'll use pre-computed embeddings from Google's Alpha Earth foundation model.

**Your Task**:
- Run the cell to load embeddings
- Note the shape of the embedding array
- Think about: What do these 64 dimensions represent?

In [ ]:
# TODO: Load foundation model embeddings for the study area for both years
# We'll load embeddings for baseline_year and target_year so we can compare features over time
# This will take around 3 minutes as we are downloading quite some data
embeddings_target = load_foundation_model_embeddings(LON_MIN, LON_MAX, LAT_MIN, LAT_MAX, target_year)

embeddings_baseline = load_foundation_model_embeddings(LON_MIN, LON_MAX, LAT_MIN, LAT_MAX, baseline_year)

print(f"Baseline embeddings ({baseline_year}) shape: {embeddings_baseline.shape}")
print(f"Target embeddings ({target_year}) shape: {embeddings_target.shape}")
print(f"  - {embeddings_target.shape[0]} feature dimensions")
print(f"  - {embeddings_target.shape[1]} x {embeddings_target.shape[2]} pixels")

### Compute Change Between Years

**TODO #3b**: Calculate the embedding difference to detect changes.

For deforestation detection, we want to identify **what changed** between the baseline and target years. We compute this by taking the difference between embeddings.

**Your Task**:
- Run the cell to compute the change embeddings
- Examine the magnitude of changes across the region
- Identify regions with high change (bright areas)
- Compare with the before/after RGB imagery
- Do the high-change areas correspond to visible deforestation?

In [ ]:
# Compute change embeddings: target - baseline
# This captures what changed between the two years
embeddings_change = embeddings_target - embeddings_baseline

print(f"Change embeddings shape: {embeddings_change.shape}")
print(f"\nChange statistics:")
print(f"  Mean absolute change: {np.abs(embeddings_change).mean():.4f}")
print(f"  Max change: {embeddings_change.max():.4f}")
print(f"  Min change: {embeddings_change.min():.4f}")
print(f"\nLarge changes may indicate:")
print("  - Deforestation (forest -> cleared land)")
print("  - Afforestation (cleared -> vegetation)")
print("  - Agricultural conversion")
print("  - Natural disturbances (fire, flooding)")

# Compute change magnitude (L2 norm across all 64 dimensions)
change_magnitude = np.linalg.norm(embeddings_change, axis=0)
print(f"\nChange magnitude range: {change_magnitude.min():.4f} to {change_magnitude.max():.4f}")

# Visualize: Baseline RGB, Target RGB, and Change Magnitude side-by-side
baseline_month = list(rgb_baseline.keys())[0]
target_month = list(rgb_target.keys())[0]

plot_rgb_with_change_magnitude(
    rgb_baseline[baseline_month], 
    rgb_target[target_month], 
    change_magnitude,
    baseline_year, 
    target_year, 
    baseline_month, 
    target_month
)

## Part 3: Unsupervised Clustering on Change Embeddings

**TODO #5**: Apply k-means clustering to identify patterns in the change embeddings.

By clustering the **change** embeddings rather than single-year embeddings, we might be able to identify different types of land cover transitions:
- Forest -> Cleared (deforestation)
- Stable forest (no change)
- Stable cleared/agriculture (no change)
- Cleared -> Regrowth (afforestation)
- Other transitions

**Your Task**:
- Run clustering with the provided k values
- **Experiment**: Try different numbers of clusters (e.g., [3, 7, 15])
- Identify which clusters represent:
  - High-change areas (likely deforestation)
  - Low-change areas (stable land cover)
  - Different types of transitions

In [ ]:
# TODO: Experiment with different numbers of clusters
number_of_clusters_to_explore = [3, 5, 10]  # Try changing this!

# Compute clusters on the CHANGE embeddings (what changed between years)
print("Computing clusters on change embeddings... (this may take a moment)")
cluster_results = compute_clusters(embeddings_change, k_values=number_of_clusters_to_explore)

# Visualize each clustering result with RGB images side-by-side
baseline_month = list(rgb_baseline.keys())[0]
target_month = list(rgb_target.keys())[0]

for k in number_of_clusters_to_explore:
    plot_rgb_with_clusters(
        rgb_baseline[baseline_month],
        rgb_target[target_month],
        cluster_results[k],
        k,
        baseline_year,
        target_year,
        baseline_month,
        target_month
    )

**Analysis Questions**:
1. What happens when you increase the number of clusters?
2. Can you identify a cluster that separates high-change from low-change areas?

## Advanced Exercise - Voluntary Homework
## Part 4: Supervised Classification - Load Labeled Data

**TODO #6**: Load reference data with embeddings from both years.

Note: This task will likely take a few hours as it requires to get familiar with Google Earth Engine and the dataset.

For supervised deforestation detection, you first require some reference data. This is the main challenge of this exercise: Curating a deforestation training-testing dataset. There are datasets on Google Earth Engine that hold data on deforestation, such as [Hansen Global Forest Change](https://developers.google.com/earth-engine/datasets/catalog/UMD_hansen_global_forest_change_2024_v1_12#bands). You will have to identify how to work with this dataset and extract a number of samples from different classes to a CSV. Choose a baseline and a target year that you are interested in (e.g 2019 and 2025 as above)

**CSV Structure:**
- Coordinates: `latitude`, `longitude`
- Land cover class: `class_name` (forest, deforested, non-forest), `label` (0, 1, 2)
- Baseline year embeddings: 64 columns named `A0_baseline`, `A1_baseline`, ..., `A63_baseline`
- Target year embeddings: 64 columns named `A0_target`, `A1_target`, ..., `A63_target`

This gives you 128 embedding columns (64 for each year) plus metadata for the same sample points.

You can build your export code based on the `export_alphaearth_embeddings_GEE.py` script

**Your Task**:
- Get familiar with GEE
- Build a reference dataset (e.g., from [Hansen Global Forest Change](https://developers.google.com/earth-engine/datasets/catalog/UMD_hansen_global_forest_change_2024_v1_12#bands))
- Randomly sample 1000 points from class that should be: forest (stable), deforested (changed in your timerange), non-forest (stable)
- Export embeddings for both baseline_year and target_year for each point
- Structure columns as: coordinates, labels, A0_baseline...A63_baseline, A0_target...A63_target
- Download CSV to `demo_data/` folder
- Run the cell below to load and examine the data

In [ ]:
# Load pre-extracted embeddings with labels from BOTH years in a single CSV
# The CSV contains AlphaEarth embeddings for the same labeled points across both years

# Update this filename to match your exported CSV
samples_csv = f"demo_data/deforestation_samples_{baseline_year}_{target_year}.csv"

print(f"Loading samples with embeddings from both {baseline_year} and {target_year}...")
print(f"File: {samples_csv}")

samples_df = pd.read_csv(samples_csv)

print(f"\nLoaded {len(samples_df)} samples")
print(f"\nColumns in CSV: {list(samples_df.columns[:10])}... (showing first 10)")
print(f"\nClass distribution:")
print(samples_df['class_name'].value_counts())
print(f"\nFirst few rows:")
print(samples_df[['longitude', 'latitude', 'class_name', 'label']].head())

### Compute Change Embeddings from CSV

**TODO #6b**: Compute change embeddings from the baseline and target columns.

Since the CSV contains embeddings from both years (suffixed with `_baseline` and `_target`), we can directly compute the change by subtracting:
- `change_embedding = target_embedding - baseline_embedding`

This creates 64-dimensional change vectors for each sample.

**Your Task**:
- Run the cell to extract baseline and target embeddings
- Compute change embeddings by subtraction
- Verify the data structure is correct

In [ ]:
# Compute change embeddings by subtracting baseline from target
# Extract embedding columns (suffixed with _baseline and _target)

# Get baseline embedding columns (A0_baseline, A1_baseline, ..., A63_baseline)
baseline_cols = [col for col in samples_df.columns if col.endswith('_baseline')]
baseline_cols.sort()  # Ensure correct order: A0, A1, ..., A63

# Get target embedding columns (A0_target, A1_target, ..., A63_target)
target_cols = [col for col in samples_df.columns if col.endswith('_target')]
target_cols.sort()  # Ensure correct order: A0, A1, ..., A63

print(f"Found {len(baseline_cols)} baseline embedding dimensions")
print(f"Found {len(target_cols)} target embedding dimensions")

if len(baseline_cols) != 64 or len(target_cols) != 64:
    raise ValueError(f"Expected 64 embedding dimensions for each year, got {len(baseline_cols)} and {len(target_cols)}")

# Extract embeddings as numpy arrays
embeddings_baseline_csv = samples_df[baseline_cols].values
embeddings_target_csv = samples_df[target_cols].values

print(f"\nBaseline embeddings shape: {embeddings_baseline_csv.shape}")
print(f"Target embeddings shape: {embeddings_target_csv.shape}")

# Compute change embeddings: target - baseline
change_features = embeddings_target_csv - embeddings_baseline_csv

print(f"\nChange feature shape: {change_features.shape}")
print(f"  - {change_features.shape[0]} samples")
print(f"  - {change_features.shape[1]} dimensions (64-d change vectors)")
print(f"\nChange statistics:")
print(f"  Mean absolute change: {np.abs(change_features).mean():.4f}")
print(f"  Max change: {change_features.max():.4f}")
print(f"  Min change: {change_features.min():.4f}")

# Add change features to dataframe for later use
samples_df['change_features'] = list(change_features)

print(f"\n Data ready for training!")

### Prepare Training and Test Splits Using Change Features

**TODO #7**: Create balanced train and test sets from the change embeddings.

We'll use the change embeddings (target - baseline) as features. This focuses the model on **what changed** rather than the state at a single time point.

**Your Task**:
- Adjust the number of training and test samples per class
- Run the cell to create the splits
- Note: We're training on **change vectors**, not single-year embeddings!

In [ ]:
# Prepare training and test splits from CHANGE embeddings
# TODO: Experiment with different numbers of training samples

NUM_SAMPLES = {
    'forest': {'label': 0, 'n_train': 20, 'n_test': 100},
    'deforested': {'label': 1, 'n_train': 20, 'n_test': 100},
    'non-forest': {'label': 2, 'n_train': 20, 'n_test': 100}
}

# Prepare train/test split on change features
np.random.seed(42)
train_indices = []
test_indices = []

for class_name, class_config in NUM_SAMPLES.items():
    class_label = class_config['label']
    n_train = class_config['n_train']
    n_test = class_config['n_test']
    
    # Get all indices for this class
    class_indices = np.where(samples_df['label'] == class_label)[0]
    
    if len(class_indices) < n_train + n_test:
        print(f"Warning: Class {class_name} has only {len(class_indices)} samples")
        n_train = min(n_train, len(class_indices) // 2)
        n_test = len(class_indices) - n_train
    
    # Sample training and test indices
    train_idx = np.random.choice(class_indices, n_train, replace=False)
    remaining = list(set(class_indices) - set(train_idx))
    test_idx = np.random.choice(remaining, min(n_test, len(remaining)), replace=False)
    
    train_indices.extend(train_idx)
    test_indices.extend(test_idx)

train_indices = np.array(train_indices)
test_indices = np.array(test_indices)
np.random.shuffle(train_indices)
np.random.shuffle(test_indices)

# Prepare X and y using change features
X_train = change_features[train_indices]
y_train = samples_df.iloc[train_indices]['label'].values
X_test = change_features[test_indices]
y_test = samples_df.iloc[test_indices]['label'].values

print(f"Training set: {len(X_train)} samples")
print(f"Test set: {len(X_test)} samples")
print(f"Feature dimension: {X_train.shape[1]} (change embeddings)")
print(f"\nClass distribution in training:")
for class_name, class_config in NUM_SAMPLES.items():
    count = np.sum(y_train == class_config['label'])
    print(f"  {class_name}: {count} samples")

### Visualize Sample Locations

**TODO #8**: Examine the spatial distribution of training and test samples.

Understanding where your samples come from is critical for:
- Detecting spatial bias
- Understanding spatial autocorrelation
- Ensuring representative coverage

**Your Task**:
- Run the cells to create interactive maps
- Examine the spatial distribution
- Are train and test samples well-separated geographically?

In [ ]:
# Prepare sample coordinates for visualization
from viz_helpers import plot_sample_map_by_class, plot_sample_map_by_split

# Add split column for visualization
samples_df['split'] = 'unused'
samples_df.iloc[train_indices, samples_df.columns.get_loc('split')] = 'train'
samples_df.iloc[test_indices, samples_df.columns.get_loc('split')] = 'test'

In [ ]:
# Visualize all samples by land cover class
m_all = plot_sample_map_by_class(samples_df, center_lat=-7.03, center_lon=-52.46, zoom=9)
m_all

In [ ]:
# Visualize train vs test split
m_split = plot_sample_map_by_split(samples_df, center_lat=-7.03, center_lon=-52.46, zoom=9)
m_split

### Train the Classifier on Change Embeddings

**TODO #9**: Train a Random Forest classifier on change features.

The model will learn to classify change patterns:
- **Forest** (class 0): Areas that remained forested
- **Deforested** (class 1): Areas where forest was cleared between baseline and target years
- **Non-forest** (class 2): Areas that were already non-forest in both years

**Your Task**:
- Run the training cell
- The model learns from **change vectors**, not static land cover
- Note how few training samples we need with foundation models!

In [ ]:
# TODO: Train a Random Forest classifier on CHANGE embeddings
print("Training Random Forest classifier on change features...")
clf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
clf.fit(X_train, y_train)

# Evaluate on test set
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print("Training complete!")
print(f"\nModel trained on {len(X_train)} samples (change embeddings)")
print(f"Test Accuracy: {accuracy:.2%}")

# Show detailed results
class_names = ['forest', 'deforested', 'non-forest']
plot_classification_results(y_test, y_pred, accuracy, class_names)

**Analysis Questions**:
1. How does the model perform on detecting deforestation (class 1)?
2. Which transitions are most confused with each other?
   - Is stable forest sometimes confused with deforestation?
   - Is stable non-forest mistaken for deforestation?
3. What might cause classification errors in change detection?
   - Seasonal vegetation changes vs. actual clearing?
   - Cloud shadows or atmospheric effects?
4. How would you improve the change detection model?
   - More training samples?
   - Additional temporal snapshots?
   - Multi-temporal features?

## Part 5: Apply Classifier to Full Study Area

**TODO #10**: Use the trained classifier to predict change patterns for every pixel.

Now we'll apply the model to classify the entire study area based on **change embeddings** into:
- Forest (green): Areas that remained forested
- Deforested (red): Areas where forest was cleared
- Non-forest (yellow): Areas that were already non-forest

**Your Task**:
- Run the prediction cell
- Examine the output map
- Compare with the RGB imagery, change magnitude map, and clustering results
- Do the deforested areas match visible clearing in the imagery?

In [ ]:
# TODO: Apply classifier to all pixels using CHANGE embeddings
print(f"Predicting deforestation for entire study area using change ({baseline_year} -> {target_year})...")
# Use CHANGE embeddings for spatial prediction (what changed between years)
predictions = predict_on_embeddings(clf, embeddings_change)

print(f"Prediction complete!")
print(f"\nChange detection results:")
class_names = ['Stable Forest', 'Deforested', 'Stable Non-forest']
for label, name in enumerate(class_names):
    count = np.sum(predictions == label)
    pct = 100 * count / predictions.size
    print(f"  {name}: {pct:.1f}%")

print(f"\nDeforestation summary: {np.sum(predictions == 1)} pixels ({100*np.sum(predictions == 1)/predictions.size:.1f}%)")
print(f"classified as deforested between {baseline_year} and {target_year}")

# Visualize predictions
plt.figure(figsize=(12, 10))
plt.imshow(predictions, cmap='RdYlGn', vmin=0, vmax=2)
plt.colorbar(label='Change Class', ticks=[0, 1, 2], 
             format=plt.FuncFormatter(lambda x, p: class_names[int(x)] if int(x) < len(class_names) else ''))
plt.title(f'Deforestation Detection: {baseline_year} -> {target_year}\n(Green=Stable Forest, Red=Deforested, Yellow=Stable Non-forest)', 
          fontsize=14, fontweight='bold')
plt.axis('off')
plt.tight_layout()
plt.show()

### Compare Supervised vs Unsupervised Results on Change

**TODO #11**: Compare supervised classification with clustering on change embeddings.

Both methods analyze the **change** between years, but:
- **Clustering**: Unsupervised, finds natural groupings in change patterns
- **Classification**: Supervised, learns from labeled examples

**Your Task**:
- Run the comparison visualization
- Analyze the differences:
  - Where do they agree on deforestation?
  - Where do they disagree?
  - Which approach better identifies clearing events?
  - Does supervised learning provide clearer deforestation boundaries?

In [ ]:
# Compare classification with clustering (both on change embeddings)
print("Comparing supervised classification with unsupervised clustering...")
print("Both methods analyze CHANGE between years, not single-year land cover")

plot_classification_vs_clustering(
    embeddings_change,
    None,
    predictions,
    {5: cluster_results[5]},  # Use k=5 clustering
    class_names=['Stable Forest', 'Deforested', 'Stable Non-forest']
)

## Part 6: Analysis and Interpretation

**TODO #12**: Analyze your results and reflect on the exercise.

**Discussion Questions**:

1. **Change Detection Approach**:
   - What types of changes does the model capture beyond deforestation?
   - How does the change magnitude relate to deforestation severity?
   - Could you detect regrowth/afforestation with this approach?

2. **Model Performance**:
   - How well does your classifier identify deforested areas from change patterns?
   - What types of errors do you observe (false positives/negatives)?
   - Are there areas with high change magnitude but low deforestation probability?
   - What might explain stable areas with subtle changes?

3. **Training Data**:
   - How many samples did you use for each change class?
   - Why do we need labels for both stable and changed areas?
   - How would multi-year training data (e.g., 2018->2020, 2020->2022) improve the model?
   - What is spatial autocorrelation and why does it matter for change detection?

4. **Supervised vs Unsupervised on Change**:
   - How do clustering results on change embeddings differ from classification?
   - Can clustering identify unexpected change types?
   - Which method better separates deforestation from natural variation?
   - Could you use clustering to identify training samples?

5. **Real-World Application**:
   - How could this approach enable near-real-time deforestation alerts?
   - What cadence (weekly, monthly, annual) would be most useful?
   - What additional validation would you need before operational deployment? Think also about generalizing in time.
   - How would you distinguish intentional clearing from natural disturbances (fire, storms)?

6. **Foundation Models for Change Detection**:
   - How do pre-trained embeddings capture temporal changes?
   - What would change if you worked with raw spectral indices (NDVI, etc.)?
   - Could the model detect subtle degradation before complete clearing?
   - How would you adapt this to other change detection tasks (urbanization, flooding)?

### Important Lessons:

1. **Change detection via embedding subtraction** - by subtracting embeddings we can capture change
2. **Foundation models capture temporal dynamics** enabling effective change detection with few samples
3. **Change magnitude maps** provide unsupervised change screening before classification
4. **Multi-temporal embeddings** (baseline + target) are essential for deforestation detection

### Resources:

- [Global Forest Watch](https://www.globalforestwatch.org/): Real-time deforestation monitoring
- [Google Earth Engine](https://earthengine.google.com/): Petabyte-scale geospatial analysis
- [Alpha Earth Model](https://arxiv.org/pdf/2507.22291): Foundation model documentation
- [Sentinel-2 Documentation](https://sentinels.copernicus.eu/web/sentinel/missions/sentinel-2): Satellite mission details
- [MapBiomas](https://mapbiomas.org/): Annual land cover maps for Brazil